In [ ]:
# ================================================================
# ETAPA 04: APRENDIZADO DE MÁQUINA - CONFIGURAÇÃO DO AMBIENTE
# ================================================================

# Fecha qualquer sessão Spark anterior para evitar conflitos de memória
# Fundamental antes de iniciar uma nova sessão de ML
try:
    spark.stop()
except:
    pass

# Instalação e configuração do ambiente Java + PySpark
# Mesmo setup da etapa de pré-processamento para garantir compatibilidade
print("Configurando ambiente Java + PySpark para Machine Learning...")
!apt-get update -qq
!apt-get install -y openjdk-17-jdk-headless -qq
!pip -q install -U pyspark==3.5.1

# Configuração das variáveis de ambiente necessárias para o Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["PATH"]  = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

# Criação de nova sessão Spark otimizada para Machine Learning
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("eixo05-ml")  # Nome específico para a etapa de ML
         .getOrCreate())

print("Spark configurado com sucesso para Machine Learning!")
print(f"Versão do Spark: {spark.version}")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Spark OK -> 3.5.1


In [ ]:
# ================================================================
# CONFIGURAÇÃO DE ACESSO AOS DADOS E VERIFICAÇÃO DO SPARK
# ================================================================

# Monta o Google Drive para acessar os dados featurizados
# force_remount=False evita remontagem desnecessária se já estiver montado
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# Verifica se a sessão Spark está ativa, senão cria uma nova
# Garante que sempre temos uma sessão válida para o processamento
from pyspark.sql import SparkSession
try:
    spark
except NameError:
    spark = SparkSession.builder.getOrCreate()

# Define o caminho base onde estão os datasets featurizados
# Mesmo diretório usado na etapa de pré-processamento
base_path = "/content/drive/MyDrive/Eixo_05/dados/"

print("Google Drive e Spark prontos para carregamento dos dados!")
print(f"Caminho dos datasets: {base_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ================================================================
# CARREGAMENTO DOS DATASETS FEATURIZADOS
# ================================================================

# Carrega os três tipos de featurização gerados na etapa anterior
# Cada dataset contém as mesmas avaliações com diferentes representações

print("Carregando datasets featurizados...")

# 1. HTF (Hashing Term Frequency) - Features esparsas baseadas em hashing
HTFfeaturizedData = spark.read.parquet(base_path + "HTFfeaturizedData")

# 2. TF-IDF - Combinação de unigramas e bigramas com pesos IDF
TFIDFfeaturizedData = spark.read.parquet(base_path + "TFIDFfeaturizedData")

# 3. Word2Vec - Embeddings semânticos densos escalados
W2VfeaturizedData = spark.read.parquet(base_path + "W2VfeaturizedData")

# Adiciona nomes aos DataFrames para identificação durante o treinamento
# Útil para logging e comparação de resultados
HTFfeaturizedData.name = "HTFfeaturizedData"
TFIDFfeaturizedData.name = "TFIDFfeaturizedData"
W2VfeaturizedData.name = "W2VfeaturizedData"

# Verifica se todos os datasets foram carregados corretamente
htf_count = HTFfeaturizedData.count()
tfidf_count = TFIDFfeaturizedData.count()
w2v_count = W2VfeaturizedData.count()

print("Datasets carregados com sucesso!")
print(f"HTF: {htf_count:,} registros")
print(f"TF-IDF: {tfidf_count:,} registros")
print(f"Word2Vec: {w2v_count:,} registros")

Contagens: 50000 50000 50000


In [ ]:
# ================================================================
# CONFIGURAÇÃO DOS ALGORITMOS DE MACHINE LEARNING E PIPELINE
# ================================================================

# Imports dos algoritmos de classificação e ferramentas de avaliação
from pyspark.ml.classification import LogisticRegression, LinearSVC, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Configuração do avaliador principal baseado em acurácia
# Métrica padrão para comparar performance entre modelos
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

def fit_with_cv(estimator, grid, train, folds=2):
    """
    Treina um modelo usando validação cruzada com grid search
    
    Args:
        estimator: algoritmo de ML a ser treinado
        grid: grade de hiperparâmetros para busca
        train: dataset de treinamento
        folds: número de folds para validação cruzada
    
    Returns:
        modelo treinado com melhores hiperparâmetros
    """
    cv = CrossValidator(
        estimator=estimator,
        estimatorParamMaps=grid,
        evaluator=evaluator,
        numFolds=folds,      # 2 folds para balancear tempo vs robustez
        parallelism=2        # Paralelização para acelerar o processo
    )
    return cv.fit(train)

def train_and_eval(dataset):
    """
    Pipeline completo de treinamento e avaliação para um dataset
    
    Processo:
    1. Divide dataset em treino (80%) e teste (20%)
    2. Detecta número de classes para escolher estratégia SVM
    3. Treina Logistic Regression com grid search
    4. Treina Linear SVC (binário) ou OneVsRest (multiclasse)
    5. Avalia ambos modelos no conjunto de teste
    6. Retorna DataFrame com resultados comparativos
    
    Args:
        dataset: DataFrame com features e labels
    
    Returns:
        DataFrame do Spark com resultados de acurácia
    """
    
    # DIVISÃO TREINO/TESTE
    # Split estratificado com seed fixa para reprodutibilidade
    train, test = dataset.randomSplit([0.8, 0.2], seed=42)
    
    # Preserva o nome do dataset para logging
    try: 
        train.name = dataset.name
    except: 
        pass

    # DETECÇÃO DO NÚMERO DE CLASSES
    # Necessário para escolher estratégia do SVM
    n_classes = dataset.select("label").distinct().count()
    print(f"Processando {getattr(dataset, 'name', 'dataset')}: {n_classes} classes detectadas")

    # ===== LOGISTIC REGRESSION =====
    # Algoritmo linear probabilístico, eficiente para classificação de texto
    print("Treinando Logistic Regression...")
    
    lr = LogisticRegression(featuresCol="features", labelCol="label")
    
    # Grid search para otimização de hiperparâmetros
    lr_grid = (ParamGridBuilder()
               .addGrid(lr.maxIter, [30])              # Iterações máximas
               .addGrid(lr.regParam, [0.0, 0.01])      # Regularização L2
               .addGrid(lr.elasticNetParam, [0.0, 0.5]) # Mistura L1/L2 (0=L2, 1=L1)
               .build())
    
    lr_model = fit_with_cv(lr, lr_grid, train)
    lr_predictions = lr_model.transform(test)
    lr_acc = evaluator.evaluate(lr_predictions) * 100.0

    # ===== LINEAR SVC (SUPPORT VECTOR MACHINE) =====
    # Algoritmo baseado em margens, robusto para alta dimensionalidade
    print("Treinando Linear SVC...")
    
    svc = LinearSVC(featuresCol="features", labelCol="label")
    
    if n_classes > 2:
        # ESTRATÉGIA MULTICLASSE: One-vs-Rest
        # Treina um classificador binário para cada classe
        print("Usando estratégia One-vs-Rest para multiclasse")
        svm_est = OneVsRest(featuresCol="features", labelCol="label", classifier=svc)
        svm_grid = ParamGridBuilder().build()  # Grid vazio para simplicidade
        svm_model = fit_with_cv(svm_est, svm_grid, train)
    else:
        # ESTRATÉGIA BINÁRIA: SVM direto
        print("Usando SVM binário direto")
        svm_grid = (ParamGridBuilder()
                    .addGrid(svc.maxIter, [50])       # Iterações máximas
                    .addGrid(svc.regParam, [0.1, 0.01]) # Parâmetro de regularização
                    .build())
        svm_model = fit_with_cv(svc, svm_grid, train)
    
    svm_predictions = svm_model.transform(test)
    svm_acc = evaluator.evaluate(svm_predictions) * 100.0

    # COMPILAÇÃO DOS RESULTADOS
    # Cria DataFrame com resultados para comparação posterior
    feat_name = getattr(dataset, "name", "features")
    results_data = [
        ("LogisticRegression", feat_name, float(f"{lr_acc:.4f}")),
        ("LinearSVC", feat_name, float(f"{svm_acc:.4f}")),
    ]
    
    print(f"Logistic Regression: {lr_acc:.2f}%")
    print(f"Linear SVC: {svm_acc:.2f}%")
    print("-" * 40)
    
    return spark.createDataFrame(results_data, ["Classifier", "Featurization", "Accuracy"])

print("Funções de treinamento configuradas!")

In [ ]:
# ================================================================
# EXECUÇÃO DO TREINAMENTO E SALVAMENTO DOS MODELOS
# ================================================================

# Imports para salvamento de modelos
import joblib
import os
import json

print("Iniciando treinamento dos modelos em todas as featurizações...")
print("=" * 60)

# Cria diretório para salvar os modelos treinados
models_path = base_path + "modelos/"
os.makedirs(models_path, exist_ok=True)

# ================================================================
# FUNÇÃO PARA TREINAR E SALVAR MODELOS
# ================================================================

def train_save_and_eval(dataset):
    """
    Pipeline completo de treinamento, salvamento e avaliação para um dataset
    
    Além do treinamento original, agora salva os modelos treinados
    para reutilização posterior na etapa de análise de resultados.
    
    Processo:
    1. Divide dataset em treino (80%) e teste (20%)
    2. Detecta número de classes para escolher estratégia SVM
    3. Treina Logistic Regression com grid search
    4. Salva modelo Logistic Regression otimizado (formato PySpark)
    5. Treina Linear SVC (binário) ou OneVsRest (multiclasse)
    6. Salva modelo SVC otimizado (formato PySpark)
    7. Avalia ambos modelos no conjunto de teste
    8. Retorna DataFrame com resultados comparativos
    
    Args:
        dataset: DataFrame com features e labels
    
    Returns:
        DataFrame do Spark com resultados de acurácia
    """
    
    # DIVISÃO TREINO/TESTE
    # Split estratificado com seed fixa para reprodutibilidade
    train, test = dataset.randomSplit([0.8, 0.2], seed=42)
    
    # Preserva o nome do dataset para logging
    try: 
        train.name = dataset.name
    except: 
        pass

    # DETECÇÃO DO NÚMERO DE CLASSES
    # Necessário para escolher estratégia do SVM
    n_classes = dataset.select("label").distinct().count()
    dataset_name = getattr(dataset, "name", "features")
    print(f"Processando {dataset_name}: {n_classes} classes detectadas")

    # ===== LOGISTIC REGRESSION =====
    # Algoritmo linear probabilístico, eficiente para classificação de texto
    print("Treinando Logistic Regression...")
    
    lr = LogisticRegression(featuresCol="features", labelCol="label")
    
    # Grid search para otimização de hiperparâmetros
    lr_grid = (ParamGridBuilder()
               .addGrid(lr.maxIter, [30])              # Iterações máximas
               .addGrid(lr.regParam, [0.0, 0.01])      # Regularização L2
               .addGrid(lr.elasticNetParam, [0.0, 0.5]) # Mistura L1/L2 (0=L2, 1=L1)
               .build())
    
    lr_cv_model = fit_with_cv(lr, lr_grid, train)
    lr_predictions = lr_cv_model.transform(test)
    lr_acc = evaluator.evaluate(lr_predictions) * 100.0
    
    # SALVAR MODELO LOGISTIC REGRESSION (FORMATO PYSPARK)
    # Extrai o melhor modelo da validação cruzada
    lr_best_model = lr_cv_model.bestModel
    lr_model_path = f"{models_path}lr_{dataset_name}"
    
    # Remove diretório se já existir (PySpark não sobrescreve)
    import shutil
    if os.path.exists(lr_model_path):
        shutil.rmtree(lr_model_path)
    
    # Salva usando formato nativo do PySpark
    lr_best_model.write().overwrite().save(lr_model_path)
    print(f"Logistic Regression salvo em: {lr_model_path}")

    # ===== LINEAR SVC (SUPPORT VECTOR MACHINE) =====
    # Algoritmo baseado em margens, robusto para alta dimensionalidade
    print("Treinando Linear SVC...")
    
    svc = LinearSVC(featuresCol="features", labelCol="label")
    
    if n_classes > 2:
        # ESTRATÉGIA MULTICLASSE: One-vs-Rest
        # Treina um classificador binário para cada classe
        print("Usando estratégia One-vs-Rest para multiclasse")
        svm_est = OneVsRest(featuresCol="features", labelCol="label", classifier=svc)
        svm_grid = ParamGridBuilder().build()  # Grid vazio para simplicidade
        svm_cv_model = fit_with_cv(svm_est, svm_grid, train)
    else:
        # ESTRATÉGIA BINÁRIA: SVM direto
        print("Usando SVM binário direto")
        svm_grid = (ParamGridBuilder()
                    .addGrid(svc.maxIter, [50])       # Iterações máximas
                    .addGrid(svc.regParam, [0.1, 0.01]) # Parâmetro de regularização
                    .build())
        svm_cv_model = fit_with_cv(svc, svm_grid, train)
    
    svm_predictions = svm_cv_model.transform(test)
    svm_acc = evaluator.evaluate(svm_predictions) * 100.0
    
    # SALVAR MODELO SVC (FORMATO PYSPARK)
    # Extrai o melhor modelo da validação cruzada
    svc_best_model = svm_cv_model.bestModel
    svc_model_path = f"{models_path}svc_{dataset_name}"
    
    # Remove diretório se já existir (PySpark não sobrescreve)
    if os.path.exists(svc_model_path):
        shutil.rmtree(svc_model_path)
    
    # Salva usando formato nativo do PySpark
    svc_best_model.write().overwrite().save(svc_model_path)
    print(f"Linear SVC salvo em: {svc_model_path}")

    # COMPILAÇÃO DOS RESULTADOS
    # Cria DataFrame com resultados para comparação posterior
    feat_name = getattr(dataset, "name", "features")
    results_data = [
        ("LogisticRegression", feat_name, float(f"{lr_acc:.4f}")),
        ("LinearSVC", feat_name, float(f"{svm_acc:.4f}")),
    ]
    
    print(f"Logistic Regression: {lr_acc:.2f}%")
    print(f"Linear SVC: {svm_acc:.2f}%")
    print("-" * 40)
    
    return spark.createDataFrame(results_data, ["Classifier", "Featurization", "Accuracy"])

# Executa treinamento para cada tipo de featurização
# Compara sistematicamente todas as combinações modelo + features
results = None

# Lista dos datasets para processar em ordem de complexidade
datasets = [HTFfeaturizedData, TFIDFfeaturizedData, W2VfeaturizedData]
dataset_names = ["HTF (Hashing)", "TF-IDF", "Word2Vec"]

for i, ds in enumerate(datasets):
    print(f"\n{i+1}/3 - Processando {dataset_names[i]}...")
    print("-" * 40)
    
    # Treina, salva e avalia ambos algoritmos no dataset atual
    r = train_save_and_eval(ds)
    
    # Mostra resultados imediatos
    r.show(truncate=False)
    
    # Acumula resultados para comparação final
    if results is None:
        results = r
    else:
        results = results.union(r)

print("\n" + "=" * 60)
print("TREINAMENTO COMPLETO - COMPARAÇÃO FINAL")
print("=" * 60)

# Mostra todos os resultados ordenados por acurácia
print("\nTodos os resultados (ordenados por acurácia):")
results.orderBy(results.Accuracy.desc()).show(truncate=False)

# Identifica e destaca a melhor combinação
best = results.orderBy(results.Accuracy.desc()).limit(1)
print("\nMELHOR COMBINAÇÃO MODELO + FEATURIZAÇÃO:")
print("=" * 50)
best.show(truncate=False)

# Coleta dados da melhor combinação para análise
best_data = best.collect()[0]
best_classifier = best_data['Classifier']
best_featurization = best_data['Featurization']
best_accuracy = best_data['Accuracy']

print(f"Algoritmo: {best_classifier}")
print(f"Featurização: {best_featurization}")
print(f"Acurácia: {best_accuracy:.2f}%")

# ================================================================
# SALVAR METADADOS DA MELHOR COMBINAÇÃO (FORMATO JSON)
# ================================================================

# Salva informações sobre a melhor combinação para uso posterior
metadata = {
    'best_classifier': best_classifier,
    'best_featurization': best_featurization,
    'best_accuracy': best_accuracy,
    'models_path': models_path,
    'all_results': [row.asDict() for row in results.collect()]
}

# Usa JSON em vez de pickle para compatibilidade
metadata_path = f"{models_path}training_metadata.json"
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"\nMetadados salvos em: {metadata_path}")

print("\n" + "=" * 60)
print("MODELOS SALVOS COM SUCESSO!")
print("=" * 60)
print(f"Total de modelos salvos: {len(datasets) * 2}")
print(f"Localização: {models_path}")
print("Formato: PySpark ML (nativo) + JSON (metadados)")
print("\nPróxima etapa: Executar 'analise_resultados.ipynb'")
print("Os modelos serão carregados automaticamente para análise detalhada.")

+------------------+-----------------+--------+
|Classifier        |Featurization    |Accuracy|
+------------------+-----------------+--------+
|LogisticRegression|HTFfeaturizedData|87.7588 |
|LinearSVC         |HTFfeaturizedData|89.0819 |
+------------------+-----------------+--------+

+------------------+-------------------+--------+
|Classifier        |Featurization      |Accuracy|
+------------------+-------------------+--------+
|LogisticRegression|TFIDFfeaturizedData|88.3446 |
|LinearSVC         |TFIDFfeaturizedData|90.2535 |
+------------------+-------------------+--------+

+------------------+-----------------+--------+
|Classifier        |Featurization    |Accuracy|
+------------------+-----------------+--------+
|LogisticRegression|W2VfeaturizedData|87.1124 |
|LinearSVC         |W2VfeaturizedData|86.9306 |
+------------------+-----------------+--------+


=== MELHOR COMBO ===
+----------+-------------------+--------+
|Classifier|Featurization      |Accuracy|
+----------+---